In [ ]:
%config InlineBackend.figure_format = 'retina'

# leaderboard

In [ ]:
%run -n main
records = parse_tsv(LEADERBOARD_LINES)
leaderboard = list(parse_leaderboard(records))

# grid

## preds

In [ ]:
# %run -n main
# task_tests = {
#     _: list(load_task(_, PUBLIC, TEST))
#     for _ in TASKS
# }

In [ ]:
# %run -n main
# for conf in GRID_CONFS:
#     for task in TASKS:
#         exp_dir = find_grid_exp_dir(EXPS_DIR, conf.id, conf.model, task)
#         if not exp_dir:
#             log(f'missing {conf.id} {conf.model} {task}')
#             continue

#         if not grid_exp_finised(exp_dir, task):
#             log(f'unfinished {conf.id} {conf.model} {task}')
#             continue
            
#         if grid_preds_exist(conf.id, task):
#             continue

#         log('id: %r, task: %r', conf.id, task)
#         preds = infer_jiant(exp_dir, task, task_tests[task], batch_size=64)
#         dump_grid_preds(conf.id, task, preds)

## eval

In [ ]:
# %run -n main
# task_targets = {
#     _: list(load_task(_, PRIVATE, TEST))
#     for _ in TASKS
# }

In [ ]:
# %run -n main
# conf_task_metrics = []
# for conf in log_progress(GRID_CONFS):
#     for task in TASKS:
#         if grid_preds_exist(conf.id, task):
#             preds = list(load_grid_preds(conf.id, task))
#             metrics = eval(task, preds, task_targets[task])
#             conf_task_metrics.append([conf.id, task, metrics])

In [ ]:
# %run -n main
# lines = format_jl(conf_task_metrics)
# dump_lines(lines, CONF_TASK_METRICS)

In [ ]:
%run -n main.py
lines = load_lines(CONF_TASK_METRICS)
conf_task_metrics = list(parse_jl(lines))

In [ ]:
conf_task_scores = [
    (id, task, select_score(task, metrics))
    for id, task, metrics in conf_task_metrics
]

## show

In [ ]:
%run -n main
task_train_sizes = {
    _: sum(1 for _ in load_task(_, PUBLIC, TRAIN))
    for _ in TASKS
    if _ != LIDIRUS
}

In [ ]:
%run -n main
show_grid_scores(leaderboard, conf_task_scores, task_train_sizes)

In [ ]:
%run -n main
tasks = TASKS
# tasks = [DANETQA, MUSERC, RUCOS, RUSSE]
# show_seed_scores(leaderboard, conf_task_scores, tasks=tasks)
show_seed_scores2(conf_task_scores, tasks=tasks)

## upload

In [ ]:
# %run -n main
# for conf in GRID_CONFS:
#     print(f'python main.py s3 sync ~/exps/{conf.id}/{conf.model} //exps/{conf.id}')

# docker

## download top exps

In [ ]:
# %run -n main
# for model in MODELS:
#     for task in TASKS:
#         if task == LIDIRUS:
#             continue

#         id = select_top_conf(task, model, conf_task_scores)
#         if not id:
#             log(f'missing {model} {task}')
#             continue
            
#         leaderboard_score = score_value(find_leaderboard_score(model, task, leaderboard))
#         grid_score = score_value(find_grid_score(id, task, conf_task_scores))
#         label = '!' if grid_score < leaderboard_score else ' '

#         print(f'{label} {leaderboard_score:0.3f} {grid_score:0.3f} {id} {task:<10} {model}')
#         print(f'python main.py s3 sync //exps/{id}/{task} exps/{model}/{task}')
#     print(f'python main.py s3 sync //exps/{id}/transformers_cache exps/{model}/transformers_cache')

## build

In [ ]:
# selected = []
# for model in MODELS:
#     for task in TASKS:
#         name = TERRA if task == LIDIRUS else task
#         dir = f'exps/{model}/{name}'
#         timestamp = path_modified(dir)
#         if timestamp.day == 29:
#             selected.append((model, task))

In [ ]:
# %run -n main
# for model in MODELS:
#     for task in TASKS:
#         if (model, task) in selected:
#             print(f'python main.py docker build exps/{model} {task} {model}-{task}')

## upload

In [ ]:
# for model in MODELS:
#     for task in TASKS:
#         print(f'python main.py docker push {model}-{task}')

## preds

In [ ]:
# for model in MODELS:
#     for task in TASKS:
#         input = task_path(task, PUBLIC, TEST, '~/data')
#         output = f'~/preds/docker/{model}/{task}.jl'
#         if not exists(expanduser(output)):
#             print(f'docker run --gpus all --interactive --rm {model}-{task} --batch-size=32 < {input} > {output}')

In [ ]:
docker_scores = []
for model in log_progress(MODELS):
    for task in TASKS:
        path = expanduser(f'~/preds/docker/{model}/{task}.jl')
        if not exists(path):
            continue

        preds = list(load_jl(path))

        path = task_path(task, PRIVATE, TEST)
        targets = list(load_jl(path))

        metrics = eval(task, preds, targets)
        score = select_score(task, metrics)
        docker_scores.append([model, task, score])

In [ ]:
%run -n main
table = show_docker_leaderboard(leaderboard, docker_scores)
table

# bench

## registry

In [ ]:
# for model in MODELS:
#     for task in TASKS:
#         for index in [1, 2, 3, 4, 5]:
#             input_size = 2000
#             batch_size = 32
# #             input_size = 1
# #             batch_size = 1
#             print(
#                 f'python main.py docker bench {model}-{task} ~/data {task} --input-size={input_size} --batch-size={batch_size} '
#                 f'> ~/benches/{model}/{task}/{input_size}_{batch_size}_{index:02d}.jl'
#             )

In [ ]:
%run -n main
bench_registry = list(list_bench_registry())

## show

In [ ]:
%run -n main
records = query_bench_registry(
    bench_registry,
    input_size=1,
    batch_size=1,
)

records = query_bench_registry(
    bench_registry,
    input_size=2000,
    batch_size=32,
    task=(TERRA),
    model=(RUBERT, RUGPT3_MEDIUM)
)

benches = [
    load_bench_registry(_)
    for _ in records
]

# task=(DANETQA, PARUS, RCB, RWSD, RUSSE, TERRA, LIDIRUS)
# task=(RUCOS, MUSERC)

# model=(RUBERT, RUBERT_CONVERSATIONAL, BERT_MULTILINGUAL)
# model=(RUGPT3_LARGE, RUGPT3_MEDIUM, RUGPT3_SMALL)

show_benches(benches)

## report

In [ ]:
%run -n main
bench_groups = list(load_group_benches(bench_registry))

### gpu ram

In [ ]:
%run -n main
show_gpu_ram_bench_report(bench_groups)

In [ ]:
%run -n main
table = show_gpu_ram_bench_report2(bench_groups)
table

In [ ]:
%run -n main
show_gpu_ram_hub_size_bench_report(bench_groups)

### time

In [ ]:
%run -n main
show_init_time_bench_report(bench_groups)

In [ ]:
%run -n main
show_proc_time_bench_report(bench_groups)

In [ ]:
%run -n main
table = show_rps_bench_report(bench_groups)
table

In [ ]:
%run -n main
show_rps_order_bench_report(bench_groups)

# score perf

In [ ]:
# %run -n main
# show_score_perf(docker_scores, leaderboard, bench_groups)